# IS-4100 Lab: Predicting NFL Team Performance Using Multiple Linear Regression (MLR)

## Objective
In this lab, you will build a multiple linear regression model to predict a team's **points scored** in a game based on various features like offensive and defensive stats from both past and current NFL seasons.

## Dataset
- Use the `nfl_data_py` or `nflreadR` package to gather NFL data from past seasons (e.g., 2019-2023) and current in-season data.
- Some key features you could look at: passing yards, rushing yards, turnovers, penalties, possession time, and team win-loss record.

## Steps

### 1. Data Preparation
- Load and clean the dataset using `nfl_data_py` or `nflreadR`. Ensure that data from both past and current seasons is available.
- Create new features, such as averaging offensive and defensive statistics over the last 3-5 games.
- Handle missing values and ensure that the data is ready for analysis.

### 2. Feature Selection
- Identify key predictors of **points scored** for each game.
- Choose at least 5-7 features, such as:
  - Passing Yards
  - Rushing Yards
  - Turnovers
  - Penalties
  - Possession Time
  - Win-Loss Record

### 3. Model Construction
- Construct a **multiple linear regression** model to predict the number of points scored by a team.
- Use statistical tests (e.g., p-values) to evaluate the significance of each predictor.
- Analyze the coefficients to understand the contribution of each feature.

### 4. Model Evaluation
- Split the dataset into **training** and **testing** sets (e.g., 80/20 split).
- Calculate evaluation metrics for your model:
  - R² (coefficient of determination)
  - Mean Absolute Error (MAE)
  - Root Mean Squared Error (RMSE)
- Compare the performance of your multiple regression model to a simple baseline.

### 5. Comparison with Simple Linear Regression
- Build a simple linear regression model that uses only **total yards gained** as the predictor.
- Compare the results of this model with the multiple linear regression model in terms of performance metrics (R², MAE, RMSE).

### 6. In-Season Prediction
- Use your model with current in-season data to predict the outcome of upcoming NFL games.
- After the games are played, compare your model’s predictions with the actual results.

## Deliverables
- A written report summarizing:
  - Model development process
  - Feature selection and their importance
  - Model performance using evaluation metrics
  - Comparisons between multiple and simple linear regression models
- Visualizations, including:
  - Residual plots
  - Predicted vs. Actual points plots
- Insights on how well your model generalizes to the current season.

## Extra Hint
I'd recommend using the data dictionary that can be found [here](https://nflreadr.nflverse.com/articles/dictionary_pbp.html) to help with understanding and looking up various variables.

## Load Data

In [ ]:
# package installs
packages_list <- c('tidyverse', 'nflfastR', 'broom', 'kableExtra', 'nflreadr', 'caret', 'plotly')
install.packages(packages_list)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘numDeriv’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘prodlim’, ‘globals’, ‘listenv’, ‘parallelly’, ‘snakecase’, ‘proxy’, ‘iterators’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘timeDate’, ‘fastrmodels’, ‘furrr’, ‘future’, ‘janitor’, ‘progressr’, ‘xgboost’, ‘svglite’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’, ‘lazyeval’, ‘crosstalk’




In [ ]:
# load in packages (2)
lapply(packages_list, library, character.only = TRUE)

In [ ]:
# play by play data
pbp <- load_pbp(2016:2024)

In [ ]:
# schedule data
schedules <- load_schedules(2016:2024)

## Data Preparation & Feature Selection

Variables to extract:
* Passing Yards
* Rushing Yards
* Offensive Penalties
* Home/Away Team
* Turnovers (interceptions & fumbles)
* Sacks Against
* Tackles for Loss Against

In [ ]:
# passing data
pbp_pass <- pbp |>
  group_by(posteam, game_id) |>
  filter(
    play_type == 'pass',
    !is.na(passer_id)
  ) |>
    mutate(
      home_team = ifelse(posteam == home_team, 1, 0)
    ) |>
    summarise(
      home_team = ifelse(is.na(sum(home_team)/sum(home_team)),0,1),
      yards_per_pass = mean(yards_gained, na.rm = TRUE),
      passing_yards = sum(yards_gained, na.rm = TRUE),
      completion_pct = sum(complete_pass, na.rm = TRUE)/n(),
      completions = sum(complete_pass),
      pass_attempts = n(),
      interceptions = sum(interception == 1, na.rm = TRUE),
      offensive_penalties = sum(penalty == TRUE & penalty_team == posteam, na.rm = TRUE),
      sacks_against = sum(sack == 1)
    )

# show passing data
head(pbp_pass)

In [ ]:
# rushing data
pbp_run <- pbp |>
  group_by(posteam, game_id) |>
  filter(
    play_type == 'run',
    !is.na(rusher_id)
  ) |>
    summarise(
      yards_per_rush = mean(yards_gained, na.rm = TRUE),
      rushing_yards = sum(yards_gained, na.rm = TRUE),
      rush_attempts = n(),
      lost_fumbles = sum(fumble_lost == 1, na.rm = TRUE),
      offensive_penalties = sum(penalty == TRUE & penalty_team == posteam),
      tfls_against = sum(tackled_for_loss == 1, na.rm = TRUE)
    )

# show rushing data
head(pbp_run)

In [ ]:
# merge into offensive_data

offensive_data <- merge(pbp_pass, pbp_run, by = c("posteam", "game_id"))

offensive_data <- offensive_data |>
  mutate(
    offensive_penalties = offensive_penalties.x + offensive_penalties.y,
    total_yards = rushing_yards + passing_yards
  ) |>
    select(-c(offensive_penalties.x, offensive_penalties.y))

head(offensive_data)

In [ ]:
# get points scored for each game
points_data <- schedules |>
  group_by(game_id) |>
  select(home_score, away_score, season)

head(points_data)

In [ ]:
# merge offensive data and points data
all_data <- merge(offensive_data, points_data, by = "game_id")

head(all_data)

In [ ]:
# update data frame points scored specific to each team
all_data <- all_data |>
  mutate(
    points_scored = ifelse(home_team == 1, home_score, away_score)
  )

head(all_data)

In [ ]:
# separate data between 2024 and other years

ad_2024 <- all_data |>
  filter(
    season == 2024
  )

all_data <- all_data |>
  filter(
    season != 2024
  )

## Model Construction
Create MLR model based on chosen features.

In [ ]:
points_scored_model <-
  lm(
    points_scored ~ 1 + passing_yards + rushing_yards + offensive_penalties + home_team + interceptions + lost_fumbles + sacks_against + tfls_against,
    data = all_data
  )

print(summary(points_scored_model))

### Interpreting the Summary

**Predictor Significance**

Each predictor except the intercept is significant because all the others have p-vlaue below 0.05.  However, the intercept is not far off at 0.065.  All the other variables have the lowest p-values given by R.


**Coefficient Interpretation**

Interceptions have the greatest impact on the amount of points a team scores since its value is -2.01, followed by fumbles lost (-1.83), and offensive penalties (-1.12).  Passing and rushing yards have the lowest coefficient values because their value is almost always larger than all other variables.  Shockingly, as the amount of tackles for loss a team takes goes up by one, they are expected to score 0.36 more points.  Also according to the model, being the home team accounts for about 0.80 points.

## Model Evaluation

### Split Data (80/20)

In [ ]:
# split data using caret package

# set seed
set.seed(100)

# split data 80/20
train_index = createDataPartition(all_data$points_scored, p = 0.8, list = FALSE)

# set training and testing data
train_data  <- all_data[train_index, ]
test_data <- all_data[-train_index, ]

# check sizes to ensure 80/20 split
print(paste("Training rows: ", nrow(train_data)))
print(paste("Testing rows: ", nrow(test_data)))

### Evaluation of Model
R², Mean Absolute Error, Root Mean Squared Error

In [ ]:
# rebuild model with training_data
points_scored_model <-
  lm(
    points_scored ~ 1 + passing_yards + rushing_yards + offensive_penalties + home_team + interceptions + lost_fumbles + sacks_against + tfls_against,
    data = train_data
  )

print(summary(points_scored_model))

In [ ]:
# make predictions using new model
preds <- predict(points_scored_model, newdata = test_data)

test_data_copy <- test_data

test_data_copy['preds'] = preds

test_data_copy['error'] = test_data_copy$points_scored - test_data_copy$preds

head(test_data_copy)

### Evaluation Metrics

In [ ]:
# r-squared
r2 <- summary(points_scored_model)$r.squared

# MAE
test_data_copy <- test_data_copy |>
  mutate(
    abs_err = ifelse(error < 0, error * -1, error)
  )

mae = mean(test_data_copy$abs_err)

# RMSE
rmse <- sqrt(mean(test_data_copy$error^2))

print(paste("R squared: ", round(r2, 3)))
print(paste("Mean absolute error: ", round(mae, 3)))
print(paste("Root Mean Squared Error: ", round(rmse, 3)))

### Interpretation of metrics
**R-squared of 0.511**: about 50% of the data's variability can be accounted for by the model.

**MAE of 5.543**: The model is off by an average of 5.543 points (less than a touchdown)

### Create Baseline Model (MLR)

Create baseline simple linear regression model based on only passing yards.

In [ ]:
# baseline model of points_scored ~ passing_yards + rushing_yards + interceptions

baseline_model = lm(points_scored ~ 1 + passing_yards + rushing_yards + interceptions, data = train_data)

summary(baseline_model)

### Compare to Baseline

In [ ]:
# add baseline predictions to data
# make predictions using new model
preds <- predict(baseline_model, newdata = test_data_copy)

test_data_copy['preds_b'] = preds

test_data_copy['error_b'] = test_data_copy$points_scored - test_data_copy$preds_b

head(test_data_copy)

In [ ]:
# baseline accuracy metrics

# r-squared
r2 <- summary(baseline_model)$r.squared

# MAE
test_data_copy <- test_data_copy |>
  mutate(
    abs_err_b = ifelse(error_b < 0, error_b * -1, error_b)
  )

mae = mean(test_data_copy$abs_err_b)

# RMSE
rmse <- sqrt(mean(test_data_copy$error_b^2))

print(paste("R squared: ", round(r2, 3)))
print(paste("Mean absolute error: ", round(mae, 3)))
print(paste("Root Mean Squared Error: ", round(rmse, 3)))

**MLR Model:**
1. R-squared: 0.513
2. MAE: 5.574
3. RMSE: 7.126

**Baseline Model:**
1. R-squared: 0.490
2. MAE: 5.656
3. RMSE: 7.207

The MLR model is better using all three of the previous metrics, but only by a slight margin.  The r-squared values is only 0.23 lower, and the MAE and RMSE are comparable.

### MLR and Baseline Plots

In [ ]:
# predicted vs. actual plot (baseline)
b_plot <- ggplot(test_data_copy, aes(x = points_scored, y = preds_b)) +
  geom_point() +
  xlim(0,60) +
  ylim(0,60) +
  # perfect fit
  geom_abline(intercept = 0, slope = 1, color = 'red', size = 1) +
  # 7 point error bars
  geom_abline(intercept = 7, slope = 1, color = 'orange', size = 1) +
  geom_abline(intercept = -7, slope = 1, color = 'orange', size = 1) +
  ggtitle("Baseline Predicted vs. Actual")


b_plot

In [ ]:
# baseline residuals
ggplot(test_data_copy, aes(x = error_b)) +
  geom_histogram() +
  xlab("Residuals") +
  ggtitle("Baseline Residuals")

In [ ]:
# predicted vs. actual (mlr)
a_plot <- ggplot(test_data_copy, aes(x = points_scored, y = preds)) +
  geom_point() +
  xlim(0,60) +
  ylim(0,60) +
  # perfect fit
  geom_abline(intercept = 0, slope = 1, color = 'red', size = 1) +
  # 7 point error bars
  geom_abline(intercept = 7, slope = 1, color = 'orange', size = 1) +
  geom_abline(intercept = -7, slope = 1, color = 'orange', size = 1) +
  ggtitle("Predicted vs. Actual")


a_plot

In [ ]:
# Residuals
ggplot(test_data_copy, aes(x = error)) +
  geom_histogram() +
  xlab("Residuals") +
  ggtitle("MLR Residual Histogram")

## Comparison with Simple Linear Regression

In [ ]:
slr_model = lm(points_scored ~ 1 + total_yards,
                data = test_data_copy)

summary(slr_model)

In [ ]:
# evaluation metrics

# add slr predictions to data
# make predictions using new model
preds <- predict(slr_model, newdata = test_data_copy)

test_data_copy['preds_s'] = preds

test_data_copy['error_s'] = test_data_copy$points_scored - test_data_copy$preds_s

head(test_data_copy)

# r-squared
r2 <- summary(slr_model)$r.squared

# MAE
test_data_copy <- test_data_copy |>
  mutate(
    abs_err_slr = ifelse(error_s < 0, error_s * -1, error_s)
  )

mae = mean(test_data_copy$abs_err_s)

# RMSE
rmse <- sqrt(mean(test_data_copy$error_s^2))

print(paste("R squared: ", round(r2, 3)))
print(paste("Mean absolute error: ", round(mae, 3)))
print(paste("Root Mean Squared Error: ", round(rmse, 3)))

### Compare All Models

**MLR Complete**
1. R-squared: 0.513
2. MAE: 5.574
3. RMSE: 7.126

**MLR Baseline**
1. R-squared: 0.490
2. MAE: 5.656
3. RMSE: 7.207

**SLR**
1. R-squared:  0.43"
2. MAE:  5.844"
3. RMSE:  7.544"


## In-Season Prediction

In [ ]:
# add baseline predictions to data
# make predictions using new model
preds <- predict(points_scored_model, newdata = ad_2024)

ad_2024['preds_cur'] = preds

ad_2024['error_cur'] = ad_2024$points_scored - ad_2024$preds_cur

head(ad_2024)

### Plot Predictions vs. Actual for 2024

In [ ]:
plot1 <- ggplot(ad_2024, aes(x = points_scored, y = preds_cur)) +
  geom_point(size = 2) +
  # perfect fit
  geom_abline(intercept = 0, slope = 1, color = 'red', size = 1) +
  # 7 point error bars
  geom_abline(intercept = 7, slope = 1, color = 'orange', size = 1) +
  geom_abline(intercept = -7, slope = 1, color = 'orange', size = 1) +
  xlim(0,50) +
  ylim(0,50) +
  xlab("Points Actual") +
  ylab("Points Predicted") +
  ggtitle("Actual vs. Predicted Points 2024")

plot1

In [ ]:
# residual plot
hist <- ggplot(ad_2024, aes(x = error_cur)) +
  geom_histogram() +
  xlab("Residual") +
  ggtitle("Histogram of Residuals")

hist

In [ ]:
# evaluation metrics
mae <- round(mean(abs(ad_2024$error_cur)),3)

rmse <- round(sqrt(mean(ad_2024$error_cur^2)),3)

print(paste("MAE: ", mae))
print(paste("RMSE: ", rmse))

Based on the MAE and RMSE, my model actually is slightly more accurate for the 2024 season than for the testing and training data of the previous seasons.

**2024 MAE:** 5.501

**2024 RMSE:** 6.838


**Test MAE:** 5.574

**Test RMSE:** 7.126